# Manual Solver Derivation

We start off by defining the primary solver variables:

\begin{align}
a &= \rho u \\
b &= \rho u^2 + p \\
c &= \rho u v
\end{align}

Next, we define a few supporting equations:

\begin{align}
p &= \frac{\gamma - 1}{\gamma} \rho T \\
H_\infty &= T + \frac{1}{2}( u^2 + v^2 )
\end{align}

Elnate $T$ from equation (4) to get:

\begin{equation}
p = \frac{\gamma - 1}{\gamma} \rho \bigl( H_\infty - \frac{1}{2}( u^2 + v^2)\bigr)
\end{equation}


ELiminating $p$ from equation (2) gives:

\begin{align}
b &= \rho u^2 \
    + \frac{\gamma - 1}{\gamma} \rho H_\infty \
    - \frac{\gamma - 1}{2\gamma} \rho u^2 \
    - \frac{\gamma - 1}{2\gamma} \rho v^2 \\
  &= \rho \left[ u^2 \bigl( 1 - \frac{\gamma - 1}{2\gamma}\bigr) \
      + \frac{\gamma - 1}{\gamma} H_\infty \
      - \frac{\gamma - 1}{2\gamma} v^2 \right]
\end{align}

Introduce a new parameter $k$ as:

\begin{equation}
k = H_\infty - \frac{v^2}{2}
\end{equation}

Substituting this parameter into equation (8) gives:

\begin{equation}
b = \rho \left[ u^2 \frac{\gamma + 1}{2\gamma} \
      + \frac{\gamma - 1}{\gamma} k \right]    
\end{equation}

Noting that $\rho = \frac{a}{u}$ we get:

\begin{equation}
b u = a \left[u^2 \frac{\gamma + 1}{2\gamma} + \frac{\gamma - 1}{\gamma} k \right]
\end{equation}

From this equation, we can generate a standard quadratic equation in $u$:

\begin{equation}
u^2 \left[ a \frac{\gamma + 1}{2\gamma}\right] \
    + u\left[ -b \right] \
    + \left[ a k \frac{\gamma - 1}{\gamma} \right]
\end{equation}

The solution to this equation is:

\begin{equation}
u = \frac{b \pm \sqrt{b^2 - 2\frac{\gamma^2-1}{\gamma^2}a^2k}}{a \frac{\gamma + 1}{\gamma}}
\end{equation}

## Using the Axial Mach Number

In the original study, the solver code solved for $M_x^2$ and not $u$ directly. The code use a second parameter $\Phi$ defined as

\begin{equation}
\Phi = \frac{2(\gamma - 1)a^2k}{\gamma b^2}
\end{equation}

\begin{align}
rad &= \sqrt{1 - \Phi - \frac{\Phi}{\gamma}} = \sqrt{1 - \frac{\Phi(\gamma + 1)}{\gamma}} \\
den &= \gamma \Phi - (\gamma - 1)
\end{align}

The equation from the solver code looks like this:

\begin{equation}
M_x^2 = \frac{1 - \Phi + rad}{den}
\end{equation}

Let's look at the radical in equation (13)
\begin{equation}
b^2 - 2\frac{\gamma^2-1}{\gamma^2}a^2k
\end{equation}

This can be written using $\Phi$ as:

\begin{equation}
b^2\Bigl(1 - \frac{\Phi(\gamma + 1)}{\gamma}\Bigr)
\end{equation}

### Validating Equation (10)

In [2]:
import sympy as sp
a,b,k,Phi,gamma = sp.symbols('a b k Phi gamma')

In [3]:
eq1 = b**2*(1 - Phi*(gamma + 1)/gamma)
eq1

b**2*(-Phi*(gamma + 1)/gamma + 1)

In [4]:
eq2 = b**2 - 2*(gamma**2 - 1)/gamma**2*a**2*k
eq2

-a**2*k*(2*gamma**2 - 2)/gamma**2 + b**2

In [5]:
eq3 = eq1 - eq2
eq3.simplify()

-Phi*b**2 - Phi*b**2/gamma + 2*a**2*k - 2*a**2*k/gamma**2

In [6]:
Phi_s = 2 * (gamma - 1)*a**2*k/(gamma*b**2)
Phi_s

a**2*k*(2*gamma - 2)/(b**2*gamma)

In [7]:
eq4 = eq3.subs(Phi,Phi_s)
eq4.simplify()

0

This result confirms that equation (19) is correct.

Therefore, we can rewrite equation (9) as follows:

\begin{align}
u &= \frac{b \pm b\sqrt{\Bigl(1 - \frac{\Phi(\gamma + 1)}{\gamma}\Bigr)}}{a \frac{\gamma + 1}{\gamma}} \\
  &= \frac{b \pm b\sqrt{\Bigl(1 - \frac{\Phi(\gamma + 1)}{\gamma}\Bigr)}}{a \frac{\gamma + 1}{\gamma}} \\
  &= \frac{b \pm b(rad)}{a\frac{\gamma+1}{\gamma}}
\end{align}

For this study, we will be keeping the first point away from the body supersonic, so the plus sign will be used.

\begin{equation}
u = \frac{b(1 + rad)}{a\frac{\gamma + 1}{\gamma}}
\end{equation}

In [8]:
rad = sp.symbols('rad')

In [11]:
equ_u = b*(1 + rad)/(a * (gamma + 1)/gamma)
equ_u

b*gamma*(rad + 1)/(a*(gamma + 1))

In [12]:
equ_u2 = equ_u*equ_u
equ_u2

b**2*gamma**2*(rad + 1)**2/(a**2*(gamma + 1)**2)

\begin{equation}
u^2 = 
\end{equation}

### Eliminating $u$ using $M_x$

The local speed of sound is given as:

\begin{align}
u_s &= \sqrt{\frac{\gamma p}{\rho}} \\
    &= \sqrt{\frac{T(\gamma - 1)}{\gamma}}
\end{align}

Using this result, we can define the axial Mach Number:

\begin{equation}
M_x = \frac{u}{u_s} = \frac{u}{\sqrt{\frac{T(\gamma-1)}{\gamma}}}
\end{equation}

or:

\begin{equation}
M_x^2 = \frac{\gamma u^2}{T(\gamma - 1)}
\end{equation}

or

\begin{equation}
T(\gamma - 1)M_x^2 = \gamma \frac{b^2 \gamma^2(rad + 1)^2}{a^2(\gamma + 1)^2}
\end{equation}

But:

\begin{equation}
T = \frac{k}{1 + M_x^2\frac{\gamma - 1}{2}}
\end{equation}

\begin{equation}
T(\gamma - 1)M_x^2 = \gamma 
\end{equation}

In [15]:
xm2 = sp.symbols('xm2')

In [17]:
T_s = k/(1 + xm2*(gamma - 1)/2)
T_s

k/(xm2*(gamma - 1)/2 + 1)

In [24]:
eq_mx = T_s*(gamma - 1)*xm2 - gamma*(b**2*gamma**2*(rad + 1)**2)/(a**2*(gamma + 1)**2)
eq_mx.ratsimp()

2*k + (-4*a**2*gamma**2*k - 8*a**2*gamma*k - 4*a**2*k - b**2*gamma**4*rad**2*xm2 - 2*b**2*gamma**4*rad*xm2 - b**2*gamma**4*xm2 + b**2*gamma**3*rad**2*xm2 - 2*b**2*gamma**3*rad**2 + 2*b**2*gamma**3*rad*xm2 - 4*b**2*gamma**3*rad + b**2*gamma**3*xm2 - 2*b**2*gamma**3)/(a**2*gamma**3*xm2 + a**2*gamma**2*xm2 + 2*a**2*gamma**2 - a**2*gamma*xm2 + 4*a**2*gamma - a**2*xm2 + 2*a**2)

In [25]:
den = gamma*Phi_s - gamma + 1
den

a**2*k*(2*gamma - 2)/b**2 - gamma + 1